In [1]:
%matplotlib inline 

from weibo.get_weibo import get_weibo
from weibo.baidu_nltk import get_sentiment_simple, get_sentiment_positive_proba, tokenize,tokenize_df

import pandas as pd
import numpy as np
import utilities.misc as um
import utilities.constants as uc
import utilities.display as ud

import time

from wordcloud import WordCloud
from scipy.misc import imread


import matplotlib.pyplot as plt

import feather

In [2]:
cookie='SINAGLOBAL=1939137473236.7695.1359786817801; wvr=6; SUBP=0033WrSXqPxfM725Ws9jqgMF55529P9D9W5GrRHKMLMbdJMQDxRPi1695JpX5KMhUgL.FoM01h-N1hBce052dJLoI0jLxK-LBKBLB-2LxK.L1KnLBoeLxKnL1heLB.BLxKqL1h.L12zLxKML1KBLBo-LxKMLB-BL1KzEShM01Btt; ALF=1612486248; SSOLoginState=1580950250; SCF=AuNRrR3OOAeW3qjtI0IQx4C0_QxU1OBOXeuhJKBl0eRSrdYHNVYBElW9RuhthsOX4_MkHOgpL4Ka6hpzzytgpxE.; SUB=_2A25zPxK7DeThGeFN41cW-CrKyDyIHXVQTQNzrDV8PUNbmtAfLUbVkW9NQ9OAjktkB3DzCHYOGBrCBwVZgxa-IWQ8; SUHB=03oVaD9G-rimrV; _s_tentry=login.sina.com.cn; Apache=2865394554053.069.1580950476493; ULV=1580950476558:10:4:4:2865394554053.069.1580950476493:1580862453173; UOR=cn.wsj.com,widget.weibo.com,www.google.com; webim_unReadCount=%7B%22time%22%3A1580970355409%2C%22dm_pub_total%22%3A1%2C%22chat_group_client%22%3A0%2C%22allcountNum%22%3A9%2C%22msgbox%22%3A0%7D'

In [3]:
path="D:\\data\\weibo\\"
dump_path="D:\\data\\weibo\\dump\\%s.feather"

In [4]:
topics=['湖北','武汉','疫情','冠状病毒','口罩','医生','野味',
file_name_dict={'冠状病毒':'guanzhuangbingdu','口罩':'kouzhao','医生':'yisheng','野味':'yewei','湖北':'hubei','武汉':'wuhan','疫情':'yiqing'}
start=pd.datetime(2019,12,1)
end=um.yesterday_date()
freq='W'
dates=pd.date_range(start,end,freq=freq)

In [5]:
collector=[]
for topic in topics:
    for date in dates:
        weibo_i=get_weibo(topic,date,cookie)
        if weibo_i is not False:
            file_name='%s_%s' % (file_name_dict[topic],date.strftime('%Y-%m-%d'))
            feather.write_dataframe(weibo_i,dump_path % (file_name))
            print ('@@@ %s finished' % (file_name))

        

Start working on topic 湖北 on 2019-12-01


C:\Users\user\Dropbox\python\DEV\weibo\get_weibo.py:95: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  ],axis=1).fillna(0)


Finish page 1
Finish page 2
Finish page 3
Finish page 4
Finish page 5
Finish page 6
Finish page 7
Page 7 is the end page
@@@ hubei_2019-12-01 finished
Start working on topic 湖北 on 2019-12-08
Finish page 1
Finish page 2
Finish page 3
Finish page 4
Finish page 5
Finish page 6
Finish page 7
Finish page 8
Page 8 is the end page
@@@ hubei_2019-12-08 finished
Start working on topic 湖北 on 2019-12-15
Finish page 1
Finish page 2
Finish page 3
Finish page 4
Finish page 5
Finish page 6
Finish page 7
Page 7 is the end page
@@@ hubei_2019-12-15 finished
Start working on topic 湖北 on 2019-12-22
Finish page 1
Finish page 2
Finish page 3
Finish page 4
Finish page 5
Finish page 6
Finish page 7
Finish page 8
Finish page 9
Finish page 10
Finish page 11
Finish page 12
Page 12 is the end page
@@@ hubei_2019-12-22 finished
Start working on topic 湖北 on 2019-12-29
Finish page 1
Finish page 2
Finish page 3
Finish page 4
Finish page 5
Finish page 6
Finish page 7
Page 7 is the end page
@@@ hubei_2019-12-29 finish

Finish page 28
Finish page 29
Finish page 30
Finish page 31
Finish page 32
Finish page 33
Finish page 34
Finish page 35
Page 35 is the end page
@@@ yiqing_2020-02-02 finished


In [ ]:
# re-collect and dump the data
all_files=um.iterate_csv(dump_path,iterate_others=[True,'.feather'])
collector=[]
for file in all_files:
    df_i=feather.read_dataframe(dump_path+'%s.feather' % (file))
    collector.append(df_i)
data=pd.concat(collector,axis=0)

# get sentiment, could take a long time to run
data['sentiment']=data['content'].map(lambda x: get_sentiment_simple(x))

In [ ]:
# save the data
feather.write_dataframe(data,path+'data.feather')

In [4]:
# get the word tokenization
data=feather.read_dataframe(path+'data.feather')

collector=[]
dates=data.groupby(['date','topic']).last().index.levels[0]
topics=data.groupby(['date','topic']).last().index.levels[1]
for date in dates:
    for topic in topics:
        words_i=tokenize_df(data[(data['topic']==topic) & (data['date']==date)],'content')
        if words_i is not False:
            words_i['date']=date
            words_i['topic']=topic
            collector.append(words_i)
            print ('finish %s on %s' % (topic,date))

words_all=pd.concat(collector,axis=0)

finish 冠状病毒 on 2019-12-01 00:00:00
finish 医生 on 2019-12-01 00:00:00
finish 口罩 on 2019-12-01 00:00:00
finish 武汉 on 2019-12-01 00:00:00
finish 湖北 on 2019-12-01 00:00:00
finish 疫情 on 2019-12-01 00:00:00
finish 野味 on 2019-12-01 00:00:00
finish 医生 on 2019-12-08 00:00:00
finish 口罩 on 2019-12-08 00:00:00
finish 武汉 on 2019-12-08 00:00:00
finish 湖北 on 2019-12-08 00:00:00
finish 疫情 on 2019-12-08 00:00:00
finish 野味 on 2019-12-08 00:00:00
finish 医生 on 2019-12-15 00:00:00
finish 口罩 on 2019-12-15 00:00:00
finish 武汉 on 2019-12-15 00:00:00
finish 湖北 on 2019-12-15 00:00:00
finish 疫情 on 2019-12-15 00:00:00
finish 野味 on 2019-12-15 00:00:00
finish 医生 on 2019-12-22 00:00:00
finish 口罩 on 2019-12-22 00:00:00
finish 武汉 on 2019-12-22 00:00:00
finish 湖北 on 2019-12-22 00:00:00
finish 疫情 on 2019-12-22 00:00:00
finish 野味 on 2019-12-22 00:00:00
finish 医生 on 2019-12-29 00:00:00
finish 口罩 on 2019-12-29 00:00:00
finish 武汉 on 2019-12-29 00:00:00
finish 湖北 on 2019-12-29 00:00:00
finish 疫情 on 2019-12-29 00:00:00
finish 野

In [ ]:
feather.write_dataframe(words_all,path+'words.feather')